In [13]:
# System
import pandas as pd
import numpy as np
from selenium import webdriver
from time import sleep

class ChromeAuto:
    def __init__(self):
        self.driver_path = '../chromedriver_win32/chromedriver.exe'
        self.options = webdriver.ChromeOptions()
        self.options.add_argument('user-data-dir=JobProfile')
        self.chrome = webdriver.Chrome(
                                        executable_path=self.driver_path,
                                        options=self.options
                                      )
        
    def acessa(self, site):
        self.chrome.get(site)
    
    def sair(self):
        self.chrome.quit()
    
    def insere_job(self, job_to_search):
        try:
            job = self.chrome.find_element_by_css_selector('#cargoDesejado')
            job.clear()
            job.send_keys(job_to_search)
        except Exception as e:
            print('Erro ao inserir nome da vaga de emprego: ',e)
            pass
    
    def insere_city(self, city_or_region):
        try:
            city = self.chrome.find_element_by_css_selector('#cidade')
            city.clear()
            city.send_keys(city_or_region)
        except Exception as e:
            print('Erro ao inserir cidade ou região: ',e)
            pass
    
    def clica_busca(self):
        try:
            search_bttn = self.chrome.find_element_by_css_selector('#formBuscaVagas > div.envolveBtnBuscaVagas.gtm-class > input')
            search_bttn.click()
        except Exception as e:
            print('Erro ao clicar no botão de busca: ',e)
            pass
        
    def one_field_extractor(self,path):
        try:
            field = self.chrome.find_element_by_xpath(path)
            field = field.text
            return field
        except:
            field = pd.NA
            return field
    
    def extractor(self,seq_num):
        """
        Finds the job description box of number seq_num, extracts it's information and returns these informations in a pandas DataFrame
        """
        field_paths = {'name':f'/html/body/div[1]/article/section/ul/li[{seq_num}]/section/div/header/h2/a',
                       'published':f'/html/body/div[1]/article/section/ul/li[{seq_num}]/section/div/header/h2/time',
                       'salary':f'/html/body/div[1]/article/section/ul/li[{seq_num}]/section/div/header/h2/div/strong[1]',
                       'n_jobs':f'/html/body/div[1]/article/section/ul/li[{seq_num}]/section/div/header/h2/div/strong[2]/span[1]',
                       'city':f'/html/body/div[1]/article/section/ul/li[{seq_num}]/section/div/header/h2/div/strong[2]/span[2]',
                       'description':f'/html/body/div[1]/article/section/ul/li[{seq_num}]/section/div/article/div[1]/div'
                      }
    
        return pd.DataFrame({'job_name':[self.one_field_extractor(field_paths['name'])],
                         'publicated_in':[self.one_field_extractor(field_paths['published'])],
                         'salary':[self.one_field_extractor(field_paths['salary'])],
                         'n_jobs':[self.one_field_extractor(field_paths['n_jobs'])],
                         'city':[self.one_field_extractor(field_paths['city'])],
                         'description':[self.one_field_extractor(field_paths['description'])]})
    
    def move_foward(self):
        """
        Moves to the next page, if there is any, and returns True. Else, return False.
        """
        try:
            a = self.chrome.find_element_by_tag_name('a.arrow.next')
            a.click()
            return True
        except:
            return False
    
    def page_scanner(self):
        """
        something
        """
        jobs_information_df = self.extractor(1)
    
        boxes_per_page = 11
        for i in range(2,boxes_per_page):
            try:
                jobs_information_df = pd.concat([jobs_information_df,self.extractor(i)]) 
            except Exception as e:
                print('Erro ao escanear a página: ',e)
                pass
    
        return jobs_information_df

    
if __name__ == '__main__':
    
    chrome = ChromeAuto()
    
    url = 'https://www.catho.com.br/vagas'
    chrome.acessa(url)
    
    job_to_search = input('Escreva a vaga procurada.')
    chrome.insere_job(job_to_search)
    
    city_or_region = input('Escreva a cidade, estado ou região de interesse.')
    chrome.insere_city(city_or_region)
    
    sleep(3)
    chrome.clica_busca()
    
    all_jobs = chrome.page_scanner()

    condition = chrome.move_foward()

    while condition == True:
        try:
            df1 = chrome.page_scanner()
            all_jobs = pd.concat([all_jobs, df1])
            sleep(1)
            condition = chrome.move_foward()
        except:
            pass
    
    sleep(3)
    #chrome.sair()
    
    #
    #self.chrome.find_element_by_css_selector

Escreva a vaga procurada.Cientista de Dados
Escreva a cidade, estado ou região de interesse.São Paulo, SP


In [14]:
all_jobs.reset_index(drop=True)

,job_name,publicated_in,salary,n_jobs,city,description
0,Cientista de Dados,"sexta, 24/04","R$ 7.000,00",Barueri-SP(1),<NA>,"Será responsável pela pesquisa, desenvolviment..."
1,Cientista de Dados,"quinta, 23/04",A Combinar,São Paulo-SP(1),<NA>,Apropriar-se dos projetos de analytics do come...
2,Cientista de Dados,"quarta, 22/04",A Combinar,São Paulo-SP(1),<NA>,"Analisar dados, construir modelos de predição ..."
3,Cientista de Dados,"quinta, 16/04",A Combinar,São Paulo-SP(1),<NA>,Linguagens Python ou R - Estatística básica: c...
4,Cientista de Dados,"segunda, 13/04",A Combinar,São Paulo-SP(1),<NA>,Description: Our Client focuses on helping cli...
...,...,...,...,...,...,...
95,NaN,NaN,NaN,NaN,NaN,NaN
96,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN
98,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
all_jobs = all_jobs.dropna(how='all').reset_index(drop=True)

In [29]:
all_jobs['job_name'].value_counts()

Analista de Business Intelligence                                                                      51
Cientista de Dados                                                                                     16
Estatístico                                                                                             8
Coordenador de Business Intelligence                                                                    3
Cientista de Dados ( Junior , Pleno e Sênior )                                                          1
Cientista de dados                                                                                      1
Cientista de Dados / Spark / Python Sênior                                                              1
Cientista de Dados / Engenheiro de Machine Learning                                                     1
Analista Big Data / Cientista de Dados                                                                  1
Cientista de Dados Sênior                     

In [30]:
all_jobs['city'].value_counts()

São Paulo-SP(1)                                 36
Barueri-SP(1)                                    9
Rio de Janeiro-RJ(1)                             4
Belo Horizonte-MG(1)                             4
Ribeirão Preto-SP(1)                             3
Brasilia-DF(1)                                   2
Brasilia-DF(2)                                   1
São Paulo-SP(3)                                  1
Brasilia-DF(3)                                   1
Vargem Grande Paulista-SP(1)                     1
Jandira-SP(1)                                    1
Salvador-BA(2)                                   1
Londrina-PR(1)                                   1
Curitiba-PR(1)                                   1
Contagem-MG(1)                                   1
Carmo-RJ(1)                                      1
Vinhedo-SP(1)                                    1
São Jose-SC(1)                                   1
Campinas-SP(1)                                   1
São Paulo-SP(1) , Rio Branco-AC

In [31]:
all_jobs['salary'].value_counts()

A Combinar                        65
De R$ 2.001,00 a R$ 3.000,00       5
R$ 7.000,00                        4
De R$ 8.001,00 a R$ 9.000,00       3
De R$ 5.001,00 a R$ 6.000,00       3
R$ 3.000,00                        2
De R$ 9.001,00 a R$ 10.000,00      2
De R$ 3.001,00 a R$ 4.000,00       2
R$ 4.000,01                        1
R$ 1.800,00                        1
R$ 3.500,00                        1
R$ 4.110,00                        1
De R$ 10.001,00 a R$ 15.000,00     1
De R$ 4.001,00 a R$ 5.000,00       1
                                   1
Name: salary, dtype: int64

In [34]:
all_jobs['description'] = all_jobs['description'].fillna(value='no_description')

In [35]:
all_jobs['python'] = all_jobs['description'].apply(lambda x: 1 if 'python' in x.lower() else 0)

In [37]:
all_jobs[all_jobs['python'] == 1]

,job_name,publicated_in,salary,n_jobs,city,description,python
3,Cientista de Dados,"quinta, 16/04",A Combinar,São Paulo-SP(1),<NA>,Linguagens Python ou R - Estatística básica: c...,1
15,Cientista de Dados / Spark / Python Sênior,"sexta, 06/03",A Combinar,1 vaga:,São Paulo-SP(1),Data Science utilizando Spark e Python. Modelo...,1
